In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle
import time
from imdb import IMDb

with open('Movie_network.gpickle', 'rb') as f:
    G_Movies = pickle.load(f)

# Initialize IMDb object
ia = IMDb()

def reconstruct_title(char_list):
    return "".join(char_list).strip()

# Function to get synopsis (prefer cache, fallback to IMDbPY)
def get_movie_synopsis(char_list):
    # Reconstruct the title
    title = reconstruct_title(char_list)
    
    # Fallback to IMDbPY
    attempts = 0
    while attempts < 3:
        try:
            movies = ia.search_movie(title)
            if movies:
                movie = movies[0]
                ia.update(movie)  # Fetch full details
                synopsis = movie.get('synopsis', [False])[0]
                if synopsis:
                    #make sure text does not have [] around it
                    synopsis = synopsis.strip('[]')
                    return synopsis
                else:
                    summary = movie.get('plot', [None])
                    #make sure text does not have [] around it
                    summary = summary[0].strip('[]') if summary else None
                    if summary != None:
                        return summary
                    else:
                        return summary, print(f"No synopsis or summary found for '{title}'.")
            else:
                return print(f"No movie found for '{title}'.")
        except Exception as e:
            attempts += 1
            time.sleep(1)
    return print(f"Failed to fetch synopsis for '{title}' after 3 attempts.")
def fetch_synopsis(node_data):
    title = node_data.get('title')
    if title:
        char_list = list(title)
        return title, get_movie_synopsis(char_list)
    return None

#make nodes in graph only 100 first nodes
#G_Movies = G_Movies.subgraph(list(G_Movies.nodes())[:100])

# Use ThreadPoolExecutor to fetch synopses concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    futures = {executor.submit(fetch_synopsis, data): node for node, data in G_Movies.nodes(data=True)}

    for future in as_completed(futures):
        node = futures[future]
        try:
            result = future.result()
            if result:
                title, synopsis = result
        except Exception as e:
            print(f"Error fetching synopsis for node {node}: {e}")
        # Store the synopsis in the node data
        G_Movies.nodes[node]['synopsis'] = synopsis

# Save the updated graph with synopses
with open('Movie_network_with_synopses.gpickle', 'wb') as f:
    pickle.dump(G_Movies, f)

